In [47]:
from datetime import date, timedelta
import pandas as pd
import numpy as np
import warp_prism
import sqlalchemy as sa
import talib
import matplotlib.pyplot as plt
import seaborn

In [48]:
metadata = sa.MetaData()
engine = sa.create_engine('postgresql://developer:1@localhost:5432/go_finance')
tbl_prices = sa.Table(
    'prices', metadata,
    sa.Column('symbol', sa.String(16)),
    sa.Column('dt', sa.Date),
    sa.Column('open', sa.FLOAT),
    sa.Column('high', sa.FLOAT),
    sa.Column('low', sa.FLOAT),
    sa.Column('close', sa.FLOAT),
    sa.Column('volume', sa.BIGINT),
    sa.Column('adj', sa.FLOAT),
)

In [49]:
symbol = 'SPY'
query = sa.select(tbl_prices.c).where(
    tbl_prices.c.symbol.in_(['SPY', 'QQQ', 'DIA', 'IWM', 'XLK', 'XLI', 'XLV', 'XLP', 'XLF', 'XLU', 'XLY', 'XLB'])
).where(
    tbl_prices.c.dt >= (date(2004, 1, 1) - timedelta(days=400)).strftime('%Y-%m-%d')
).order_by('symbol', 'dt')

is_adjusted = False
df = warp_prism.to_dataframe(query, bind=engine)

fltr = df['dt'] >= '2004-01-01'
print(df[fltr].iloc[0])

df.head()

symbol                    DIA
dt        2004-01-02 00:00:00
open                   105.04
high                   105.47
low                    103.99
close                  104.37
volume                8231300
adj                   73.1989
Name: 275, dtype: object


,symbol,dt,open,high,low,close,volume,adj
0,DIA,2002-11-27,87.65,89.65,87.59,89.53,8264700,61.399529
1,DIA,2002-11-29,89.70,89.80,88.82,88.90,3058100,60.967476
2,DIA,2002-12-02,90.40,90.66,88.03,88.87,8928700,60.946902
3,DIA,2002-12-03,88.20,88.39,87.42,87.93,7011500,60.302252
4,DIA,2002-12-04,86.94,88.40,86.70,87.75,9947600,60.178808


In [50]:
# adjust
if not is_adjusted:
    ratio = df.adj / df.close
    df['open'] *= ratio
    df['high'] *= ratio
    df['low'] *= ratio
    df['close'] *= ratio
    df['volume'] = (df['volume'] / ratio).astype(int)
    is_adjusted = True
df.head()

,symbol,dt,open,high,low,close,volume,adj
0,DIA,2002-11-27,60.110228,61.481825,60.069080,61.399529,12051209,61.399529
1,DIA,2002-11-29,61.516115,61.584695,60.912612,60.967476,4459182,60.967476
2,DIA,2002-12-02,61.996174,62.174481,60.370832,60.946902,13019424,60.946902
3,DIA,2002-12-03,60.487417,60.617719,59.952495,60.302252,10223850,60.302252
4,DIA,2002-12-04,59.623311,60.624577,59.458720,60.178808,14505137,60.178808


## ROCs

In [51]:
period = 200
roc = talib.ROC(df['close'].values, timeperiod=period)
df['roc200'] = roc
df['roc200_smooth'] = (df['roc200'] + df['roc200'].shift(1)*2 + df['roc200'].shift(2)*2 + df['roc200'].shift(3)) / 6 
df.tail()

,symbol,dt,open,high,low,close,volume,adj,roc200,roc200_smooth
47311,XLY,2018-07-23,112.13,112.25,111.420,112.03,3369156,112.03,24.080719,25.358769
47312,XLY,2018-07-24,112.71,112.89,111.440,111.70,4825617,111.70,23.026564,24.474337
47313,XLY,2018-07-25,111.39,112.56,111.060,112.52,6269782,112.52,23.780974,23.808886
47314,XLY,2018-07-26,112.40,112.79,111.691,111.93,5802016,111.93,23.522596,23.536398
47315,XLY,2018-07-27,113.41,113.57,111.140,111.64,6548826,111.64,23.323958,23.492944


## TSI

In [52]:
def get_tsi(df, short, long):
    ratio = (df.close - df.close.shift(short)).abs() / talib.ATR(df.high.values, df.low.values, df.close.values, timeperiod=short)
    tsi = talib.SMA(talib.SMA(ratio.values, timeperiod=short), timeperiod=long) - 1  # substruct 1
    tsi[tsi < 0] = 0
    #print("Min: {0} Max: {1}".format(tsi[~np.isnan(tsi)].min(), tsi[~np.isnan(tsi)].max()))    
    return tsi

df['tsi'] = get_tsi(df, 10, 50)  # 10, 50

for_sign = df.roc200.values.copy()
df['tsi'] *= np.sign(for_sign)  # separate bull/bear trends

/home/alex/env/lib/python3.6/site-packages/ipykernel_launcher.py:4: RuntimeWarning: invalid value encountered in less
  after removing the cwd from sys.path.


## KST
https://stockcharts.com/school/doku.php?id=chart_school:technical_indicators:know_sure_thing_kst

In [53]:
def get_kst(df, roc, sma, signal):
    kst = None
    for i, v in enumerate(roc):
        data = talib.SMA(talib.ROC(df.close.values, timeperiod=v), timeperiod=sma[i])
        if kst is None:
            kst = data * (i + 1)
        else:
            kst += data * (i + 1)
    return kst, talib.SMA(kst, timeperiod=signal)

df['kst'], df['kst_signal'] = get_kst(df, [10, 15, 20, 30], [10, 10, 10, 15], 9)

In [54]:
def get_drawdown(series, is_prod=False):
    total = len(series.index)
    if is_prod:
        series = series.rolling(total, min_periods=1).apply(np.prod)
    rolling_max = series.rolling(total, min_periods=1).max()
    daily_drawdown = series/rolling_max - 1.0
    daily_drawdown_max = daily_drawdown.rolling(total, min_periods=1).min()
    return daily_drawdown_max.min()

def get_strategy_returns(df, symbol='-'):
    fltr = df['dt'] >= '2004-01-01'
    calc_returns = lambda x: (np.prod(x) - 1) * 100
    
    # get price changes
    chg = (df.close / df.close.shift(1))[fltr]
    #chg = (df.open / df.open.shift(1))[fltr]  # look in the future
    chg.iloc[0] = 1  # set first value as 1

    benchmark = chg
    sma50x200_chg = chg[(df.close.rolling(50).mean() - df.close.rolling(200).mean()).shift(1) >= 0]
    
    roc200 = talib.ROC(df.close.values, timeperiod=200)
    roc200_chg = chg[np.roll(roc200, 1)[fltr] >= 0]

    roc5_200 = talib.SMA(talib.ROC(df.close.values, timeperiod=5), timeperiod=200)
    roc5_200_chg = chg[np.roll(roc5_200, 1)[fltr] >= 0]
    
    #roc_n_sma_chg = chg[(np.roll(roc5_200, 1)[fltr] >= 0) & ((df.close.rolling(50).mean() - df.close.rolling(200).mean()).shift(1)[fltr] >= 0)]
    
    tsi = get_tsi(df, 10, 50)  # 10, 50
    for_sign = roc200.copy()
    tsi *= np.sign(for_sign)
    tsi_chg = chg[np.roll(tsi, 1)[fltr] >= 0.5]
    
    kst, kst_signal = get_kst(df, [10, 15, 20, 30], [10, 10, 10, 15], 9)
    kst_chg = chg[np.roll(kst, 1)[fltr] >= 0]
    kst_hist_chg = chg[np.roll(kst - kst_signal, 1)[fltr] >= 0]  # KST histogram
    
    returns = {
        'symbol': symbol, 
        'bench': calc_returns(benchmark),
        'bench dd': get_drawdown(benchmark, is_prod=True) * 100,
        's50x200': calc_returns(sma50x200_chg[fltr]),
        's50x200 dd': get_drawdown(sma50x200_chg[fltr], is_prod=True) * 100,
        'roc200': calc_returns(roc200_chg[fltr]),
        'roc200 dd': get_drawdown(roc200_chg[fltr], is_prod=True) * 100,
        'roc5_200': calc_returns(roc5_200_chg[fltr]),
        'roc5_200 dd': get_drawdown(roc5_200_chg[fltr], is_prod=True) * 100,
        #'roc_n_sma': calc_returns(roc_n_sma_chg[fltr]),
        #'roc_n_sma dd': get_drawdown(roc_n_sma_chg[fltr], is_prod=True) * 100,
        'tsi': calc_returns(tsi_chg[fltr]),
        'tsi dd': get_drawdown(tsi_chg[fltr], is_prod=True) * 100,
        'kst': calc_returns(kst_chg[fltr]),
        'kst dd': get_drawdown(kst_chg[fltr], is_prod=True) * 100,
        'kst_hist': calc_returns(kst_hist_chg[fltr]),
        'kst_hist dd': get_drawdown(kst_hist_chg[fltr], is_prod=True) * 100,
    }
    return returns

l = []
for symbol in df.symbol.unique():
    l.append(get_strategy_returns(df[df.symbol == symbol], symbol))
    
print("Returns & Drawdowns")
pd.DataFrame(l).sort_values('symbol')[['symbol', 'bench', 'bench dd', 'kst', 'kst dd', 'kst_hist', 'kst_hist dd', 'roc5_200', 'roc5_200 dd', 'tsi', 'tsi dd', 's50x200', 's50x200 dd']]

/home/alex/env/lib/python3.6/site-packages/ipykernel_launcher.py:4: RuntimeWarning: invalid value encountered in less
  after removing the cwd from sys.path.


Returns & Drawdowns


,symbol,bench,bench dd,kst,kst dd,kst_hist,kst_hist dd,roc5_200,roc5_200 dd,tsi,tsi dd,s50x200,s50x200 dd
0,DIA,247.354851,-51.869349,94.866839,-25.470484,79.516615,-32.180564,218.316542,-15.890008,121.649002,-23.339370,242.748382,-23.709626
1,IWM,258.913269,-59.045272,111.519806,-23.441446,82.590255,-36.432430,137.391678,-41.275912,33.991080,-44.077232,128.121948,-37.805471
2,QQQ,449.218472,-53.403956,208.051720,-22.154957,117.071428,-28.539935,256.020820,-24.784662,138.361446,-26.660181,222.623899,-26.049942
3,SPY,238.332215,-55.189440,98.607274,-20.936348,56.547646,-26.053776,191.722567,-24.014941,135.056120,-28.424293,237.676603,-19.179015
4,XLB,209.011891,-59.829151,43.722548,-29.612336,-12.696455,-53.159670,168.478212,-34.064910,44.813308,-37.773269,152.955855,-24.955137
5,XLF,66.413270,-82.687240,116.326459,-42.124130,6.312191,-63.467154,148.811551,-44.913329,61.479027,-32.182785,192.172400,-30.285786
6,XLI,278.841496,-62.263194,133.494030,-27.689191,76.829362,-31.635653,215.812233,-35.103519,158.836387,-24.242790,202.025751,-30.012292
7,XLK,327.467382,-53.039291,137.611009,-30.311501,106.761040,-28.902633,156.023519,-26.538146,75.377768,-27.989749,158.568303,-24.582904
8,XLP,255.414106,-32.390335,103.072797,-16.003010,71.986552,-18.065247,252.752179,-11.941026,58.504207,-14.037877,124.168412,-26.185478
9,XLU,281.781259,-46.476601,126.746837,-18.260207,63.979698,-24.827260,269.822640,-15.041804,138.280005,-13.183036,205.222869,-23.358325
